<a href="https://colab.research.google.com/github/JMartinezArribas/Tutoriales/blob/main/Regresion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import sys, os

ON_COLAB = 'google.colab' in sys.modules

if ON_COLAB:
    UCI_ROOT = 'https://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/'
    os.system(f'wget {UCI_ROOT}/auto-mpg.data')
    os.system(f'wget {UCI_ROOT}/auto-mpg.names')

#%run -i setup.py

In [18]:
with open("auto-mpg.names") as f:
    print(f.read())

1. Title: Auto-Mpg Data

2. Sources:
   (a) Origin:  This dataset was taken from the StatLib library which is
                maintained at Carnegie Mellon University. The dataset was 
                used in the 1983 American Statistical Association Exposition.
   (c) Date: July 7, 1993

3. Past Usage:
    -  See 2b (above)
    -  Quinlan,R. (1993). Combining Instance-Based and Model-Based Learning.
       In Proceedings on the Tenth International Conference of Machine 
       Learning, 236-243, University of Massachusetts, Amherst. Morgan
       Kaufmann.

4. Relevant Information:

   This dataset is a slightly modified version of the dataset provided in
   the StatLib library.  In line with the use by Ross Quinlan (1993) in
   predicting the attribute "mpg", 8 of the original instances were removed 
   because they had unknown values for the "mpg" attribute.  The original 
   dataset is available in the file "auto-mpg.data-original".

   "The data concerns city-cycle fuel consumptio

In [139]:
columns = ['mpg','cylinders','displacement','horsepower','weight','acceleration',\
          'model year','origin','car name']

df = pd.read_fwf('auto-mpg.data',header=None,names=columns)

df.head()
  

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name
0,18.0,8,307.0,130.0,3504.0,12.0,70,1,"""chevrolet chevelle malibu"""
1,15.0,8,350.0,165.0,3693.0,11.5,70,1,"""buick skylark 320"""
2,18.0,8,318.0,150.0,3436.0,11.0,70,1,"""plymouth satellite"""
3,16.0,8,304.0,150.0,3433.0,12.0,70,1,"""amc rebel sst"""
4,17.0,8,302.0,140.0,3449.0,10.5,70,1,"""ford torino"""


In [140]:
#Eliminamos las comillas que aparecen en el campo de la marca del coche ("car name")
df['car name'] = df['car name'].str.replace('"','')

In [141]:
df.sample(10,random_state=123)

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name
150,26.0,4,108.0,93.00,2391.0,15.5,74,3,subaru
42,12.0,8,383.0,180.0,4955.0,11.5,71,1,dodge monaco (sw)
120,19.0,4,121.0,112.0,2868.0,15.5,73,2,volvo 144ea
102,26.0,4,97.0,46.00,1950.0,21.0,73,2,volkswagen super beetle
293,31.9,4,89.0,71.00,1925.0,14.0,79,2,vw rabbit custom
264,18.1,8,302.0,139.0,3205.0,11.2,78,1,ford futura
54,35.0,4,72.0,69.00,1613.0,18.0,71,3,datsun 1200
299,27.2,4,141.0,71.00,3190.0,24.8,79,2,peugeot 504
309,41.5,4,98.0,76.00,2144.0,14.7,80,2,vw rabbit
91,13.0,8,400.0,150.0,4464.0,12.0,73,1,chevrolet caprice classic


In [91]:
# Este dataset tiene 398 registros para 8 variables, 4 de tipo continuo, 3 discreto
# y 2 de tipo categórico
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           398 non-null    float64
 1   cylinders     398 non-null    int64  
 2   displacement  398 non-null    float64
 3   horsepower    398 non-null    object 
 4   weight        398 non-null    float64
 5   acceleration  398 non-null    float64
 6   model year    398 non-null    int64  
 7   origin        398 non-null    int64  
 8   car name      398 non-null    object 
dtypes: float64(4), int64(3), object(2)
memory usage: 28.1+ KB


In [94]:
# Los caballos de potencia aparecen como categórica aunque sean un valor númerico
# Saquemos un resumen de los principales estadísticos de los valores númericos
df.describe().T

,count,mean,std,min,25%,50%,75%,max
mpg,398.0,23.514573,7.815984,9.0,17.500,23.0,29.000,46.6
cylinders,398.0,5.454774,1.701004,3.0,4.000,4.0,8.000,8.0
displacement,398.0,193.425879,104.269838,68.0,104.250,148.5,262.000,455.0
weight,398.0,2970.424623,846.841774,1613.0,2223.750,2803.5,3608.000,5140.0
acceleration,398.0,15.568090,2.757689,8.0,13.825,15.5,17.175,24.8
model year,398.0,76.010050,3.697627,70.0,73.000,76.0,79.000,82.0
origin,398.0,1.572864,0.802055,1.0,1.000,1.0,2.000,3.0


In [142]:
# Saquemos un resumen de los principales estadísticos de los valores categóricos
df[['horsepower', 'car name']].describe(include='O').T

,count,unique,top,freq
horsepower,398,94,150.0,22
car name,398,305,ford pinto,6


In [143]:
# Horsepower debemos formatearla a otro tipo de dato
df['horsepower'] = df.horsepower.astype(float)

ValueError: ignored

In [144]:
# Parece que hay algún caracter('?')en esta columna y por eso pandas al importarlo lo convirtió a objeto
mask = pd.to_numeric(df['horsepower'], errors='coerce').isna()
df[mask]

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name
32,25.0,4,98.0,?,2046.0,19.0,71,1,ford pinto
126,21.0,6,200.0,?,2875.0,17.0,74,1,ford maverick
330,40.9,4,85.0,?,1835.0,17.3,80,2,renault lecar deluxe
336,23.6,4,140.0,?,2905.0,14.3,80,1,ford mustang cobra
354,34.5,4,100.0,?,2320.0,15.8,81,2,renault 18i
374,23.0,4,151.0,?,3035.0,20.5,82,1,amc concord dl


In [145]:
# Obligamos a cambiar los '?' a NaN para imputar.
df['horsepower'] = pd.to_numeric(df['horsepower'], errors='coerce')

In [148]:
df[mask]

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name
32,25.0,4,98.0,NaN,2046.0,19.0,71,1,ford pinto
126,21.0,6,200.0,NaN,2875.0,17.0,74,1,ford maverick
330,40.9,4,85.0,NaN,1835.0,17.3,80,2,renault lecar deluxe
336,23.6,4,140.0,NaN,2905.0,14.3,80,1,ford mustang cobra
354,34.5,4,100.0,NaN,2320.0,15.8,81,2,renault 18i
374,23.0,4,151.0,NaN,3035.0,20.5,82,1,amc concord dl


In [149]:
from sklearn.impute import KNNImputer

df_imp = KNNImputer().fit_transform(df[['mpg','cylinders','displacement','horsepower','weight','acceleration',\
          'model year','origin']])

In [159]:
df_imp = pd.DataFrame(data=df_imp, columns= columns[:-1])
df_imp[mask]

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin
32,25.0,4.0,98.0,62.0,2046.0,19.0,71.0,1.0
126,21.0,6.0,200.0,107.6,2875.0,17.0,74.0,1.0
330,40.9,4.0,85.0,64.6,1835.0,17.3,80.0,2.0
336,23.6,4.0,140.0,112.8,2905.0,14.3,80.0,1.0
354,34.5,4.0,100.0,76.0,2320.0,15.8,81.0,2.0
374,23.0,4.0,151.0,88.2,3035.0,20.5,82.0,1.0


In [164]:
df[(df.weight>=2000) & (df.weight<=2100)]


,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name
32,25.0,4,98.0,NaN,2046.0,19.0,71,1,ford pinto
51,30.0,4,79.0,70.0,2074.0,19.5,71,2,peugeot 304
52,30.0,4,88.0,76.0,2065.0,14.5,71,2,fiat 124b
84,27.0,4,97.0,88.0,2100.0,16.5,72,3,toyota corolla 1600 (sw)
145,32.0,4,83.0,61.0,2003.0,19.0,74,3,datsun 710
151,31.0,4,79.0,67.0,2000.0,16.0,74,2,fiat x1.9
195,29.0,4,85.0,52.0,2035.0,22.2,76,1,chevrolet chevette
216,31.5,4,98.0,68.0,2045.0,18.5,77,3,honda accord cvcc
237,30.5,4,98.0,63.0,2051.0,17.0,77,1,chevrolet chevette
238,33.5,4,98.0,83.0,2075.0,15.9,77,1,dodge colt m/m


In [127]:
# En la descripción del dataset nos aparece que efectivamente hay 6 registros '?'
# en horsepower.
df.sample(10,random_state=123)

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name,origin_cat
150,26.0,4,108.0,93.00,2391.0,15.5,74,3,subaru,Asia
42,12.0,8,383.0,180.0,4955.0,11.5,71,1,dodge monaco (sw),América
120,19.0,4,121.0,112.0,2868.0,15.5,73,2,volvo 144ea,Europa
102,26.0,4,97.0,46.00,1950.0,21.0,73,2,volkswagen super beetle,Europa
293,31.9,4,89.0,71.00,1925.0,14.0,79,2,vw rabbit custom,Europa
264,18.1,8,302.0,139.0,3205.0,11.2,78,1,ford futura,América
54,35.0,4,72.0,69.00,1613.0,18.0,71,3,datsun 1200,Asia
299,27.2,4,141.0,71.00,3190.0,24.8,79,2,peugeot 504,Europa
309,41.5,4,98.0,76.00,2144.0,14.7,80,2,vw rabbit,Europa
91,13.0,8,400.0,150.0,4464.0,12.0,73,1,chevrolet caprice classic,América


In [115]:
df2 = df[~mask]

In [116]:
df2['horsepower'] = df2.horsepower.astype(float)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [118]:
df2['horsepower'].value_counts()

150.0    22
90.0     20
88.0     19
110.0    18
100.0    17
         ..
94.0      1
167.0     1
158.0     1
137.0     1
107.0     1
Name: horsepower, Length: 93, dtype: int64

In [126]:
# La variable origin está codificada según la región de procedencia del modelo, siendo:
# 1 = America
# 2 = Europa
# 3 = Asia
# Podemos crear una variable para descodificar esta variable para que nos sea
# más descriptiva en los gráficos:

origin_mapping = {1: 'América', 2:'Europa', 3:'Asia'}
df['origin_cat'] = df['origin'].apply(lambda x: origin_mapping[x])

df['origin_cat'].value_counts()

América    249
Asia        79
Europa      70
Name: origin_cat, dtype: int64

In [108]:
X_train = train_df[['cylinders', 'displacement', 'weight',
       'acceleration', 'model year', 'origin']].values
y_train = train_df['mpg'].values

X_test = test_df[['cylinders', 'displacement', 'weight',
       'acceleration', 'model year', 'origin']].values
y_test = test_df['mpg'].values

NameError: ignored